In [6]:
%reload_ext autoreload
%autoreload 2

from os import makedirs
from os.path import exists
from shutil import copy
from typing import Any, Dict, List, Tuple
from sqlite3 import connect
from baboon_tracking import BaboonTracker
from library.config import set_config_part
from library.dataset import get_dataset_path, get_dataset_list
from library.region import bb_intersection_over_union

from sherlock import Sherlock

import pickle
import numpy as np
import pandas as pd
import yaml

runtime_config = {"display": False, "save": True, "timings": False}

# video_files = get_dataset_list("VISO")
video_files = [
    "VISO/car/003", # Video 1
    # "VISO/car/009", # Video 7
]

In [9]:
def extend(target: Dict[str, Any], source: Dict[str, Any]):
    for key, value in source.items():
        target[key] = value

    return target

def get_config_declaration(root: str, config_declaration: Dict[str, Any]):
    leaf_nodes = {}

    for key, value in config_declaration.items():
        if isinstance(value, dict):
            extend(leaf_nodes, get_config_declaration(f"{root}/{key}", value))
            continue

    if not leaf_nodes.keys():
        leaf_nodes[root] = config_declaration

    return leaf_nodes

def get_config_options(config_declaration: Dict[str, Any]):
    type_value = None
    if config_declaration["type"] == "int32":
        type_value = np.int32
    elif config_declaration["type"] == "float":
        type_value = np.float32

    if "step" in config_declaration:
        step = config_declaration["step"]
    else:
        step = 1

    if "min" in config_declaration:
        min_value = config_declaration["min"]
    else:
        min_value = 0

    if "max" in config_declaration:
        max_value = config_declaration["max"]
    else:
        max_value = 100

    if "odd" in config_declaration:
        is_odd = config_declaration["odd"]
    else:
        is_odd = False

    if is_odd:
        min_value -= 1
        max_value = max_value / 2

    values = np.arange(min_value, max_value, step=step, dtype=type_value)
    if is_odd:
        values = values * 2 + 1

    return values

def get_metrics(results_db_path: str, ground_truth_path: str):
    with connect(results_db_path) as connection:
        cursor = connection.cursor()

        ground_truth = pd.read_csv(ground_truth_path).to_numpy()
        found_regions = cursor.execute(
            "SELECT x1, y1, x2, y2, frame FROM motion_regions"
        )

        curr_frame = -1
        true_positive = 0
        false_negative = 0
        false_positive = 0
        truth = None
        for x1, y1, x2, y2, frame in found_regions:
            if curr_frame != frame:
                if truth is not None:
                    false_negative += truth.shape[0]

                truth = np.array(
                    [
                        (x1, y1, x1 + width, y1 + height)
                        for x1, y1, width, height in ground_truth[
                            ground_truth[:, 0] == frame, 2:6
                        ]
                    ]
                )
                curr_frame = frame

            current = (x1, y1, x2, y2)
            matches = np.array([bb_intersection_over_union(current, t) for t in truth])

            if np.sum(matches) > 0:
                true_positive += 1
                truth = truth[np.logical_not(matches == np.max(matches))]
            else:
                false_positive += 1

    if false_negative + true_positive == 0:
        recall = 0
    else:
        recall = true_positive / (false_negative + true_positive)

    if true_positive + false_positive == 0:
        precision = 0
    else:
        precision = true_positive / (true_positive + false_positive)

    if recall + precision == 0:
        f1 = 0
    else:
        f1 = (2 * recall * precision) / (recall + precision)
    
    return recall, precision, f1

def get_score(X: np.ndarray, y: np.ndarray, known_idx: np.ndarray, video_file: str, config_options: List[Tuple[str, np.ndarray]]):
    if exists("./score_cache.pickle"):
        with open("./score_cache.pickle", "rb") as f:
            score_cache = pickle.load(f)
    else:
        score_cache = {}

    if video_file not in score_cache:
        score_cache[video_file] = {}
    video_score_cache = score_cache[video_file]

    path = get_dataset_path(video_file)
    ground_truth_path = f"./data/Datasets/{video_file}/gt/gt.txt"
    for idx in known_idx:
        hash_str = str(X[idx, :])

        if hash_str not in video_score_cache:
            for i, (key, _, value_type) in enumerate(config_options):
                config_value = X[idx, i]
                if value_type == "int32":
                    config_value = int(config_value)

                set_config_part(key, config_value)

            baboon_tracker = BaboonTracker(path, runtime_config=runtime_config)
            baboon_tracker.run(20)

            recall, precision, f1 = get_metrics("./output/results.db", ground_truth_path)
            video_score_cache[hash_str] = (recall, precision, f1)

            with open("./score_cache.pickle", "wb") as f:
                pickle.dump(score_cache, f)

        recall, precision, f1 = video_score_cache[hash_str]
        y[idx, :] = np.array([recall, precision])

        print(f"Completed: {idx} with Recall: {recall} Precision: {precision} F1: {f1}")

for video_file in video_files:
    with open("./config_declaration.yml", "r") as f:
        config_declaration = get_config_declaration("", yaml.safe_load(f))

    config_options = [
        (k, get_config_options(i), i["type"])
        for k, i in config_declaration.items()
        if "skip_learn" not in i or not i["skip_learn"]
    ]
    X = np.array(np.meshgrid(*[c for _, c, _ in config_options])).T.reshape(-1, len(config_options))
    y = np.zeros((X.shape[0], 2))

    sherlock = Sherlock(
        n_init=5,
        budget=int(X.shape[0]*0.15),
        surrogate_type="rbfthin_plate-rbf_multiquadric-randomforest-gpy",
        kernel="matern",
        num_restarts=0,
        pareto_margin=0,
        y_hint=None,
        plot_design_space=False,
        use_ted_in_loop=False,
        request_output=lambda y, idx: get_score(X, y, idx, video_file, config_options),
        action_only=None,
        n_hint_init=0,
        scale_output=True,
        use_trace_as_prior=True,
        model_selection_type="mab10"
    )

    sherlock.fit(X).predict(X, y)

    print(sherlock.known_idx)

  0%|          | 0/20 [00:00<?, ?it/s]Process ForkPoolWorker-119:
Process ForkPoolWorker-132:
Process ForkPoolWorker-123:
Process ForkPoolWorker-126:
Process ForkPoolWorker-121:
Process ForkPoolWorker-124:
Process ForkPoolWorker-130:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-127:
Traceback (most recent call last):
Process ForkPoolWorker-118:
Process ForkPoolWorker-131:
Process ForkPoolWorker-122:
Process ForkPoolWorker-125:
Process ForkPoolWorker-128:
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/pool.py", line 692, in _terminate_pool
    cls._help_stuff_finish(inqueue, task_handler, len(pool))
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/pool.py

Process ForkPoolWorker-102:
Process ForkPoolWorker-112:
Process ForkPoolWorker-114:
Traceback (most recent call last):
Process ForkPoolWorker-110:
Process ForkPoolWorker-108:
Process ForkPoolWorker-105:
Process ForkPoolWorker-103:
Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-109:
Process ForkPoolWorker-113:
Process ForkPoolWorker-115:
Process ForkPoolWorker-106:
Process ForkPoolWorker-104:
Traceback (most recent call last):
Process ForkPoolWorker-111:
Process ForkPoolWorker-107:
Traceback (most recent call last):
Process ForkPoolWorker-116:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-101:
Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
  

Process ForkPoolWorker-89:
Process ForkPoolWorker-100:
Process ForkPoolWorker-88:
Process ForkPoolWorker-90:
Process ForkPoolWorker-91:
Traceback (most recent call last):
Process ForkPoolWorker-94:
Process ForkPoolWorker-99:
Process ForkPoolWorker-86:
Process ForkPoolWorker-85:
Process ForkPoolWorker-87:
Process ForkPoolWorker-98:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-97:
Process ForkPoolWorker-93:
Process ForkPoolWorker-92:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-96:
Traceback (most recent call last):
Process ForkPoolWorker-95:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Tr

In [ ]:
for video_file in video_files:
    results_path = f"./output/results/{video_file}/results.db"
    if exists(results_path):
        continue

    path = get_dataset_path(video_file)

    baboon_tracker = BaboonTracker(path, runtime_config=runtime_config)
    baboon_tracker.run()

    makedirs(f"./output/results/{video_file}", exist_ok=True)
    copy("./output/results.db", results_path)

In [ ]:
for video_file in video_files:
    results_db_path = f"./output/results/{video_file}/results.db"
    ground_truth_path = f"./data/Datasets/{video_file}/gt/gt.txt"

    recall, precision, f1 = get_metrics(results_db_path, ground_truth_path)

    print(f"Recall: {recall}, Precision: {precision}, F1: {f1}")